In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup as bs

In [5]:
soup = bs(data.text)

In [6]:
standings_table = soup.select('table.stats_table')[0]

In [7]:
links = standings_table.find_all('a')

In [8]:
links = [l.get("href") for l in links]

In [9]:
links = [l for l in links if '/squads/' in l]

In [10]:
team_urls= [f"https://fbref.com{l}" for l in links]

In [11]:
team_url = team_urls[0]
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbre

In [12]:
data = requests.get(team_url)

In [13]:
import pandas as pd

In [14]:
matches = pd.read_html(data.text, match="Scores & Fixtures")

In [15]:
soup = bs(data.text)

In [16]:
links = soup.find_all('a')

In [17]:
links = [l.get("href") for l in links]

In [18]:
links = list(set([l for l in links if l and 'matchlogs/all_comps/' in l]))

In [19]:
links.sort()

In [20]:
links

['/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/defense/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/gca/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/keeper/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/misc/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/passing_types/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/possession/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/schedule/Manchester-City-Scores-and-Fixtures-All-Competitions',
 '/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [21]:
data_shooting = requests.get(f"https://fbref.com{links[8]}")
shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
shooting.columns = shooting.columns.droplevel()

In [22]:
data_gca = requests.get(f"https://fbref.com{links[1]}")
gca = pd.read_html(data_gca.text,match="Goal and Shot Creation")[0]
gca.columns = gca.columns.droplevel()

In [23]:
data_pt = requests.get(f"https://fbref.com{links[5]}")
pt = pd.read_html(data_pt.text,match="Pass Types")[0]
pt.columns = pt.columns.droplevel()

In [24]:
data_poss = requests.get(f"https://fbref.com{links[6]}")
poss = pd.read_html(data_poss.text,match="Possession")[0]
poss.columns = poss.columns.droplevel()

In [25]:
data_def = requests.get(f"https://fbref.com{links[0]}")
defence = pd.read_html(data_def.text,match="Defensive Actions")[0]
defence.columns = defence.columns.droplevel()

In [26]:
data_pass = requests.get(f"https://fbref.com{links[4]}")
passing = pd.read_html(data_pass.text,match="Passing")[0]
passing.columns = passing.columns.droplevel()

In [27]:
data_gk = requests.get(f"https://fbref.com{links[2]}")
gk = pd.read_html(data_gk.text,match="Goalkeeping")[0]
gk.columns = gk.columns.droplevel()

In [28]:
data_misc = requests.get(f"https://fbref.com{links[3]}")
misc = pd.read_html(data_misc.text,match="Miscellaneous Stats")[0]
misc.columns = misc.columns.droplevel()

In [39]:
team_data = pd.merge(
    pd.merge(
        pd.merge(
            pd.merge(
                pd.merge(
                    pd.merge(
                        pd.merge(
                            pd.merge(matches[0],shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date"),
                            gca[["Date","SCA","GCA"]],on="Date"),
                        pt[["Date","Att","Cmp"]], on="Date"),
                    poss[["Date","Poss","Def Pen","Def 3rd","Mid 3rd","Att 3rd","Att Pen", "PrgC", "1/3","CPA"]], on="Date"),
                defence[["Date","Tkl","Blocks","Int"]], on="Date"),
            passing[["Date","KP","1/3","PrgP"]], on="Date"),
        gk[["Date","Saves"]], on="Date"),
    misc[["Date","CrdY","CrdR","Fls","Fld","Off"]], on="Date")

In [40]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Int,KP,1/3_y,PrgP,Saves,CrdY,CrdR,Fls,Fld,Off
0,2021-08-14,17:30,Premier League,Matchweek 1,Sat,Home,L,0,3,Liverpool,...,9,11.0,23.0,29.0,3,1,0,4,14,2
1,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Away,L,0,5,Manchester City,...,8,0.0,5.0,13.0,0,0,0,7,13,3
2,2021-08-24,19:00,EFL Cup,Second round,Tue,Home,W,6,0,Bournemouth,...,7,NaN,NaN,NaN,1,1,0,10,9,1
3,2021-08-28,15:00,Premier League,Matchweek 3,Sat,Home,L,1,2,Leicester City,...,15,8.0,16.0,26.0,1,2,0,8,9,2
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,L,0,1,Arsenal,...,8,7.0,18.0,26.0,5,2,0,11,9,0


In [41]:
years = list(range(2023,2021,-1))

In [42]:
all_matches = []

In [43]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [45]:
import time

for year in years: 
    data = requests.get(standings_url)
    soup = bs(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls= [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")
        
        soup = bs(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = list(set([l for l in links if l and 'matchlogs/all_comps/' in l]))
        links.sort()
        time.sleep(2)
        data_shooting = requests.get(f"https://fbref.com{links[8]}")
        shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        time.sleep(2)
        data_gca = requests.get(f"https://fbref.com{links[1]}")
        gca = pd.read_html(data_gca.text,match="Goal and Shot Creation")[0]
        gca.columns = gca.columns.droplevel()
        time.sleep(2)
        data_pt = requests.get(f"https://fbref.com{links[5]}")
        pt = pd.read_html(data_pt.text,match="Pass Types")[0]
        pt.columns = pt.columns.droplevel()
        time.sleep(2)
        data_poss = requests.get(f"https://fbref.com{links[6]}")
        poss = pd.read_html(data_poss.text,match="Possession")[0]
        poss.columns = poss.columns.droplevel()
        time.sleep(2)
        data_def = requests.get(f"https://fbref.com{links[0]}")
        defence = pd.read_html(data_def.text,match="Defensive Actions")[0]
        defence.columns = defence.columns.droplevel()
        time.sleep(2)
        data_pass = requests.get(f"https://fbref.com{links[4]}")
        passing = pd.read_html(data_pass.text,match="Passing")[0]
        passing.columns = passing.columns.droplevel()
        time.sleep(2)
        data_gk = requests.get(f"https://fbref.com{links[2]}")
        gk = pd.read_html(data_gk.text,match="Goalkeeping")[0]
        gk.columns = gk.columns.droplevel()
        time.sleep(2)
        data_misc = requests.get(f"https://fbref.com{links[3]}")
        misc = pd.read_html(data_misc.text,match="Miscellaneous Stats")[0]
        misc.columns = misc.columns.droplevel()
        time.sleep(2)
        try:
            team_data = pd.merge(
                pd.merge(
                    pd.merge(
                        pd.merge(
                            pd.merge(
                                pd.merge(
                                    pd.merge(
                                        pd.merge(matches[0],shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date"),
                                        gca[["Date","SCA","GCA"]],on="Date"),
                                    pt[["Date","Att","Cmp"]], on="Date"),
                                poss[["Date","Poss","Def Pen","Def 3rd","Mid 3rd","Att 3rd","Att Pen", "PrgC", "1/3","CPA"]], on="Date"),
                            defence[["Date","Tkl","Blocks","Int"]], on="Date"),
                        passing[["Date","KP","1/3","PrgP"]], on="Date"),
                    gk[["Date","Saves"]], on="Date"),
                misc[["Date","CrdY","CrdR","Fls","Fld","Off"]], on="Date")
        except ValueError("No tables found"):
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

In [46]:
match_df = pd.concat(all_matches)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")

In [ ]:
match_df